In [1]:
# Imports
import numpy as np
import pandas as pd
import pulp

from warnings import filterwarnings
filterwarnings("ignore")


vehicles_df = pd.read_csv('../data/Master_data/vehicle_master_data.csv')
distances_df = pd.read_csv('../data/Master_data/distance_master_data.csv')

In [2]:
input_df = pd.read_csv("../data/Input_data/Input2.csv")

In [3]:
vehicles = {m:[{i:vehicles_df[i][j] for i in (vehicles_df[vehicles_df['Transportation modes']==m].columns[1:])} for j in range(len(vehicles_df[vehicles_df['Transportation modes']==m]))] for m in vehicles_df['Transportation modes']}
distances = {(distances_df['Source'][j], distances_df['Destination'][j]):{i:distances_df[i][j] for i in distances_df.columns[2:]} for j in range(len(distances_df))}

df = distances_df.join(vehicles_df, how="cross")
df['decisions'] = df['Source'] + ', ' + df['Destination'] + ', ' + df['Transportation modes'] + ', ' + df['Vehicle_type']

In [4]:
df

,Source,Destination,Air Distance (km),Rail Distance (km),Road Distance (km),Transportation modes,Vehicle_type,Capacity (metric tons),Co2e (g/km),Avg Speed (km/hr),Cost per km,decisions
0,Mumbai,Kolkata,1655.79,1723.03,2071.24,Air,CARGO PLANE,100,551.584454,500,10.00,"Mumbai, Kolkata, Air, CARGO PLANE"
1,Mumbai,Kolkata,1655.79,1723.03,2071.24,Rail,GOODS RAIL,5000,97.565914,50,4.00,"Mumbai, Kolkata, Rail, GOODS RAIL"
2,Mumbai,Kolkata,1655.79,1723.03,2071.24,Rail,MIXED RAIL,8000,107.090591,90,5.50,"Mumbai, Kolkata, Rail, MIXED RAIL"
3,Mumbai,Kolkata,1655.79,1723.03,2071.24,Road,HGV DROPSIDE LORRY,24,156.598994,45,1.30,"Mumbai, Kolkata, Road, HGV DROPSIDE LORRY"
4,Mumbai,Kolkata,1655.79,1723.03,2071.24,Road,4WD PICK UP,18,155.605476,42,1.20,"Mumbai, Kolkata, Road, 4WD PICK UP"
...,...,...,...,...,...,...,...,...,...,...,...,...
375,Chennai,Bangalore,290.95,303.61,365.20,Road,HGV (Small),15,190.411547,52,1.43,"Chennai, Bangalore, Road, HGV (Small)"
376,Chennai,Bangalore,290.95,303.61,365.20,Road,LARGE VAN,8,145.588808,60,1.44,"Chennai, Bangalore, Road, LARGE VAN"
377,Chennai,Bangalore,290.95,303.61,365.20,Road,MOTOR CARAVAN,13,125.195841,50,1.47,"Chennai, Bangalore, Road, MOTOR CARAVAN"
378,Chennai,Bangalore,290.95,303.61,365.20,Road,HGV LOADER,4,186.902781,28,1.37,"Chennai, Bangalore, Road, HGV LOADER"


In [5]:
master_availability = dict(zip(df['Vehicle_type'].unique(), (10,)*len(df['Vehicle_type'].unique())))

master_demand = {}
for i, j in zip(np.array(df[['Source', 'Destination']].drop_duplicates()), [0,]*len(df[['Source', 'Destination']].drop_duplicates())):
    master_demand[tuple(i)] = j

In [6]:
demand = {}
for i, j in zip(np.array(input_df[['From', 'To']]), input_df['Quantity (MT)']):
    demand[tuple(i)] = j
    
for k in master_demand.keys():
    if k not in demand.keys():
        demand[k] = 0

In [7]:
%%time

##########################################################
#------------------- Model Definition -------------------#
##########################################################
# Create a LP minimization problem
model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)
            
############################################################
#------------------- Decision Variables -------------------#
############################################################

choice_decision =  pulp.LpVariable.dicts("Choice_Decision_", (tuple(df['decisions'][i].split(", ")) + (j,)
                      for i in range(len(df)) for j in range(master_availability[df['decisions'][i].split(", ")[-1]])),
                                  lowBound=0, cat="Binary")

############################################################
#------------------- Objective Function -------------------#
############################################################

obj_tup = [choice_decision[(s, d, m, v, c)]\
    *df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()\
    *df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Co2e (g/km)'].item()/1000
          for (s, d, m, v, c) in choice_decision.keys()]
model += pulp.lpSum(obj_tup)


#####################################################
#------------------- Constraints -------------------#
#####################################################

#========== Demand Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for (s, d, m, v, c) in choice_decision.keys():
    if(s0, d0) == (s, d):
        col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
    else:
        if demand[(s0, d0)] == 0:
            model += (pulp.lpSum(col)==demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        elif demand[(s0, d0)] >= 0:
            model += (pulp.lpSum(col)>=demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())

if demand[(s0, d0)] == 0:
    model += (pulp.lpSum(col)==demand[(s0, d0)])
elif demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col)>=demand[(s0, d0)])
    

#========== Time Constraint ==========

for (s, d, m, v, c) in choice_decision.keys():
    if input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
        exp = choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()/df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Avg Speed (km/hr)'].item()
        model += pulp.LpConstraint(e=pulp.LpAffineExpression(exp), sense=pulp.LpConstraintLE, rhs = input_df[(input_df['From']==s) & (input_df['To']==d)]['Time (hrs)'].item())

#========== Cost Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for (s, d, m, v, c) in choice_decision.keys():
    if((s0, d0) == (s, d)) & input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
        col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Cost per km'].item())
    elif (s0, d0) != (s, d):
        if input_df[(input_df['From']==s0) & (input_df['To']==d0)].shape[0]:
            model += (pulp.lpSum(col)<=input_df[(input_df['From']==s0) & (input_df['To']==d0)]['Cost'])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        else:
            (s0, d0) = (s, d)
            col=[]

if input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
    model += (pulp.lpSum(col)<=input_df[(input_df['From']==s0) & (input_df['To']==d0)]['Cost'])

CPU times: total: 17.5 s
Wall time: 17.6 s


In [8]:
%%time
###############################################
#------------------- Solve -------------------#
###############################################

model.solve()
print(pulp.LpStatus[model.status])

Optimal
CPU times: total: 78.1 ms
Wall time: 209 ms


In [9]:
#######################################################################
#------------------- Mappings Necessary for Output -------------------#
#######################################################################

mt = []
for v in model.variables():
    if  v.varValue != 0:
        mt.append(v.name.split('__(')[1].rstrip(')').split(',_') + [v.varValue,])
        
mt_output = pd.DataFrame(mt, columns=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Count'])
mt_output_df = mt_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_output_df['S_D_M'] = mt_output_df['Source'] + '_' + mt_output_df['Destination'] + '_' + mt_output_df['Transportation mode']

dist_list = []
vals = []
for c in distances_df.columns[2:]:
    for i in range(len(distances_df)):
        dist_list.append(distances_df['Source'][i] + '_' + distances_df['Destination'][i] + '_' + c.split(' ')[0])
        vals.append(distances_df[c][i])
        
distance_mappings = dict(zip(dist_list, vals))
quantity_mappings = dict(zip(vehicles_df['Vehicle_type'], vehicles_df['Capacity (metric tons)']))
emission_mappings = dict(zip(vehicles_df['Vehicle_type'], vehicles_df['Co2e (g/km)']))
cost_mappings = dict(zip(vehicles_df['Vehicle_type'], vehicles_df['Cost per km']))

mt_output_df['Loaded quantity'] = mt_output_df['Vehicle type'].str.replace('_', ' ').map(quantity_mappings)
mt_output_df['Distance'] = mt_output_df['S_D_M'].map(distance_mappings)
mt_output_df['Emission'] = (mt_output_df['Vehicle type'].str.replace('_', ' ').map(emission_mappings))*mt_output_df['Distance']/1000
mt_output_df['Transportation cost'] = (mt_output_df['Vehicle type'].str.replace('_', ' ').map(cost_mappings))*mt_output_df['Distance']

In [10]:
###########################################################
#------------------- Generating Output -------------------#
###########################################################

vehicle_count = pd.pivot_table(mt_output_df, values='Count', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()
vehicle_count['Total Vehicles in Use'] = vehicle_count.iloc[:, 2:].sum(axis=1)


material_transport_quantity = pd.pivot_table(mt_output_df, values='Loaded quantity', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()
material_transport_quantity['Total Quantity Transported'] = material_transport_quantity.iloc[:, 2:].sum(axis=1)


co2_emission = pd.pivot_table(mt_output_df, values='Emission', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()
co2_emission['Total co2 Emission (kg)'] = co2_emission.iloc[:, 2:].sum(axis=1)



transportation_cost = pd.pivot_table(mt_output_df, values='Transportation cost', index=['Source', 'Destination'],
                       columns=['Vehicle type'], aggfunc='sum').reset_index()
transportation_cost['Total transportation cost'] =  transportation_cost.iloc[:, 2:].sum(axis=1)

vehicle_count.fillna(0, inplace=True)
material_transport_quantity.fillna(0, inplace=True)
co2_emission.fillna(0, inplace=True)
transportation_cost.fillna(0, inplace=True)

In [11]:
vehicle_count

Vehicle type,Source,Destination,4WD_PICK_UP,CARGO_PLANE,GOODS_RAIL,MIXED_RAIL,Total Vehicles in Use
0,Bangalore,Chennai,1.0,0.0,0.0,0.0,1.0
1,Bangalore,Kolkata,0.0,0.0,0.0,2.0,2.0
2,Bangalore,Mumbai,0.0,1.0,0.0,0.0,1.0
3,Chennai,Bangalore,0.0,0.0,1.0,4.0,5.0
4,Chennai,Delhi,0.0,0.0,1.0,0.0,1.0
5,Chennai,Kolkata,0.0,0.0,2.0,0.0,2.0
6,Chennai,Mumbai,0.0,0.0,1.0,0.0,1.0
7,Delhi,Chennai,0.0,0.0,1.0,0.0,1.0
8,Delhi,Kolkata,0.0,0.0,0.0,3.0,3.0
9,Delhi,Mumbai,0.0,0.0,1.0,0.0,1.0


In [12]:
material_transport_quantity

Vehicle type,Source,Destination,4WD_PICK_UP,CARGO_PLANE,GOODS_RAIL,MIXED_RAIL,Total Quantity Transported
0,Bangalore,Chennai,18.0,0.0,0.0,0.0,18.0
1,Bangalore,Kolkata,0.0,0.0,0.0,16000.0,16000.0
2,Bangalore,Mumbai,0.0,100.0,0.0,0.0,100.0
3,Chennai,Bangalore,0.0,0.0,5000.0,32000.0,37000.0
4,Chennai,Delhi,0.0,0.0,5000.0,0.0,5000.0
5,Chennai,Kolkata,0.0,0.0,10000.0,0.0,10000.0
6,Chennai,Mumbai,0.0,0.0,5000.0,0.0,5000.0
7,Delhi,Chennai,0.0,0.0,5000.0,0.0,5000.0
8,Delhi,Kolkata,0.0,0.0,0.0,24000.0,24000.0
9,Delhi,Mumbai,0.0,0.0,5000.0,0.0,5000.0


In [13]:
co2_emission

Vehicle type,Source,Destination,4WD_PICK_UP,CARGO_PLANE,GOODS_RAIL,MIXED_RAIL,Total co2 Emission (kg)
0,Bangalore,Chennai,56.82712,0.000000,0.000000,0.000000,56.827120
1,Bangalore,Kolkata,0.00000,0.000000,0.000000,347.361183,347.361183
2,Bangalore,Mumbai,0.00000,464.588554,0.000000,0.000000,464.588554
3,Chennai,Bangalore,0.00000,0.000000,29.621987,130.055097,159.677085
4,Chennai,Delhi,0.00000,0.000000,177.737777,0.000000,177.737777
5,Chennai,Kolkata,0.00000,0.000000,275.237347,0.000000,275.237347
6,Chennai,Mumbai,0.00000,0.000000,104.754571,0.000000,104.754571
7,Delhi,Chennai,0.00000,0.000000,177.737777,0.000000,177.737777
8,Delhi,Kolkata,0.00000,0.000000,0.000000,437.405094,437.405094
9,Delhi,Mumbai,0.00000,0.000000,116.130756,0.000000,116.130756


In [14]:
transportation_cost

Vehicle type,Source,Destination,4WD_PICK_UP,CARGO_PLANE,GOODS_RAIL,MIXED_RAIL,Total transportation cost
0,Bangalore,Chennai,438.24,0.0,0.00,0.00,438.24
1,Bangalore,Kolkata,0.00,0.0,0.00,17839.91,17839.91
2,Bangalore,Mumbai,0.00,8422.8,0.00,0.00,8422.80
3,Chennai,Bangalore,0.00,0.0,1214.44,6679.42,7893.86
4,Chennai,Delhi,0.00,0.0,7286.88,0.00,7286.88
5,Chennai,Kolkata,0.00,0.0,11284.16,0.00,11284.16
6,Chennai,Mumbai,0.00,0.0,4294.72,0.00,4294.72
7,Delhi,Chennai,0.00,0.0,7286.88,0.00,7286.88
8,Delhi,Kolkata,0.00,0.0,0.00,22464.42,22464.42
9,Delhi,Mumbai,0.00,0.0,4761.12,0.00,4761.12


In [15]:
vehicle_count.to_csv('../data/Output_data/v2/Vehicle_Count.csv', index=False)
material_transport_quantity.to_csv('../data/Output_data/v2/Material_Transport_Quantity.csv', index=False)
co2_emission.to_csv('../data/Output_data/v2/CO2_Emission.csv', index=False)
transportation_cost.to_csv('../data/Output_data/v2/Transportation_Cost.csv', index=False)